In [5]:
from numpy import *
from itertools import combinations, permutations

In [6]:
sequence = sorted([
    (31, 'E'), (32, 'D'), (33, 'F'), (35, 'A'), (37, 'B'), (38, 'C'), (39, 'E'),
    (40, 'F'), (42, 'C'), (44, 'D'), (46, 'B'), (47, 'A'), (48, 'D'), (50, 'C'),
    (53, 'E'), (54, 'F'), (55, 'C'), (57, 'B'), (58, 'E'), (59, 'A'), (60, 'E'),
    (61, 'C'), (62, 'F'), (65, 'A'), (67, 'D'),
], key=lambda x:x[0])

In [7]:
def createC1(sequence, step):
    C1 = {}
    for event in sequence:
        mo = tuple((event[0], event[0] + step)) # calc minimal occurrence (mo)
        evt1 = tuple(event[1])
        if evt1 in C1: # if event exists in C1
            if mo not in C1[evt1]: # if mo not exists in event's set of mo
                C1[evt1].add(mo)
        else: C1[evt1] = set((mo,))
    return C1

In [8]:
C1 = createC1(sequence, 1)
C1

{('E',): {(31, 32), (39, 40), (53, 54), (58, 59), (60, 61)},
 ('D',): {(32, 33), (44, 45), (48, 49), (67, 68)},
 ('F',): {(33, 34), (40, 41), (54, 55), (62, 63)},
 ('A',): {(35, 36), (47, 48), (59, 60), (65, 66)},
 ('B',): {(37, 38), (46, 47), (57, 58)},
 ('C',): {(38, 39), (42, 43), (50, 51), (55, 56), (61, 62)}}

In [9]:
def createLargeEpisodeSet(Ck, min_fr):
    Lk = Ck.copy()
    for episode in Ck:
        if len(Ck[episode]) < min_fr: # if number of mo < min_fr
            del Lk[episode]
    return Lk

In [10]:
L1 = createLargeEpisodeSet(C1, 2)
L1

{('E',): {(31, 32), (39, 40), (53, 54), (58, 59), (60, 61)},
 ('D',): {(32, 33), (44, 45), (48, 49), (67, 68)},
 ('F',): {(33, 34), (40, 41), (54, 55), (62, 63)},
 ('A',): {(35, 36), (47, 48), (59, 60), (65, 66)},
 ('B',): {(37, 38), (46, 47), (57, 58)},
 ('C',): {(38, 39), (42, 43), (50, 51), (55, 56), (61, 62)}}

In [11]:
def checkSubsetFrequency(candidate, Lk, k):
    if k > 1:    
        subsets = list(combinations(candidate, k))
    else:
        return True
    for elem in subsets:
        if not elem in Lk: #elem is tuple
            return False
    return True

In [18]:
def createCandidates(Lk, L1, k, max_width):
    # create candidates list
    candidatesK = [] #candidates list
    generatedCans = [] 
    lkKeys = sorted(set([item for t in Lk for item in t])) # get all episodes and sort / Lk:get all keys
    generatedCans = list(permutations(lkKeys, k))
    for can in generatedCans:
        if checkSubsetFrequency(can, Lk, k-1): #if all subsets of can is frequent / Lk:get all keys
            candidatesK.append(can)
    # create minimal occurrence of candidates
    if len(candidatesK) == 0: # if candidates list is null
        return dict()
    else:
        Ck = {}
        if k == 2: #candidate 2
            for can in candidatesK: # can is tuple
                Ck[can] = set()
                for mok in Lk[tuple(can[0])]:
                    for mo1 in L1[tuple(can[1])]:
                        # mo1.ts >= mok.te and mo1.te - mok.ts <= max_width
                        if mo1[0] >= mok[1] and mo1[1] - mok[0] <= max_width:
                            Ck[can].add(tuple((mok[0], mo1[1])))
        else: #candidate > 2
            for can in candidatesK: # can is tuple
                Ck[can] = set()
                for mok in Lk[can[:-1]]:
                    for mo1 in L1[tuple(can[-1])]:
                        # mo1.ts >= mok.te and mo1.te - mok.ts <= max_width
                        if mo1[0] >= mok[1] and mo1[1] - mok[0] <= max_width:
                            Ck[can].add(tuple((mok[0], mo1[1])))
        # recheck minimal occurrence is minimal
        for episode, setOfMOs in Ck.items():
            rem = set() #remove set
            for mo1 in setOfMOs:
                for mo2 in setOfMOs.difference(mo1):
                    # mo1[0] is mo1.ts, mo2[0] is mo2.ts, mo1[1] is mo1.te, mo2[1] is mo2.te
                    if (mo1[0]==mo2[0] and mo1[1] < mo2[1]) or (mo1[0] > mo2[0] and mo1[1] == mo2[1]) or (mo1[0] > mo2[0] and mo1[1] < mo2[1]):
                        rem.add(mo2)
            Ck[episode] = setOfMOs.difference(rem)
        return Ck

In [15]:
C2 = createCandidates(L1, L1, 2, 5)
C2

{('A', 'B'): {(35, 38)},
 ('A', 'C'): {(35, 39), (47, 51), (59, 62)},
 ('A', 'D'): {(47, 49), (65, 68)},
 ('A', 'E'): {(35, 40), (59, 61)},
 ('A', 'F'): {(59, 63)},
 ('B', 'A'): {(46, 48), (57, 60)},
 ('B', 'C'): {(37, 39), (46, 51), (57, 62)},
 ('B', 'D'): {(46, 49)},
 ('B', 'E'): {(37, 40), (57, 59)},
 ('B', 'F'): {(37, 41)},
 ('C', 'A'): {(55, 60), (61, 66)},
 ('C', 'B'): {(42, 47), (55, 58)},
 ('C', 'D'): {(42, 45)},
 ('C', 'E'): {(38, 40), (50, 54), (55, 59)},
 ('C', 'F'): {(38, 41), (50, 55), (61, 63)},
 ('D', 'A'): {(32, 36), (44, 48)},
 ('D', 'B'): {(44, 47)},
 ('D', 'C'): {(48, 51)},
 ('D', 'E'): set(),
 ('D', 'F'): {(32, 34)},
 ('E', 'A'): {(31, 36), (58, 60)},
 ('E', 'B'): {(53, 58)},
 ('E', 'C'): {(39, 43), (53, 56), (60, 62)},
 ('E', 'D'): {(31, 33)},
 ('E', 'F'): {(31, 34), (39, 41), (53, 55), (60, 63)},
 ('F', 'A'): {(33, 36), (62, 66)},
 ('F', 'B'): {(33, 38), (54, 58)},
 ('F', 'C'): {(40, 43), (54, 56)},
 ('F', 'D'): {(40, 45)},
 ('F', 'E'): {(54, 59)}}

In [16]:
L2 = createLargeEpisodeSet(C2, 2)
L2

{('A', 'C'): {(35, 39), (47, 51), (59, 62)},
 ('A', 'D'): {(47, 49), (65, 68)},
 ('A', 'E'): {(35, 40), (59, 61)},
 ('B', 'A'): {(46, 48), (57, 60)},
 ('B', 'C'): {(37, 39), (46, 51), (57, 62)},
 ('B', 'E'): {(37, 40), (57, 59)},
 ('C', 'A'): {(55, 60), (61, 66)},
 ('C', 'B'): {(42, 47), (55, 58)},
 ('C', 'E'): {(38, 40), (50, 54), (55, 59)},
 ('C', 'F'): {(38, 41), (50, 55), (61, 63)},
 ('D', 'A'): {(32, 36), (44, 48)},
 ('E', 'A'): {(31, 36), (58, 60)},
 ('E', 'C'): {(39, 43), (53, 56), (60, 62)},
 ('E', 'F'): {(31, 34), (39, 41), (53, 55), (60, 63)},
 ('F', 'A'): {(33, 36), (62, 66)},
 ('F', 'B'): {(33, 38), (54, 58)},
 ('F', 'C'): {(40, 43), (54, 56)}}

In [19]:
C3 = createCandidates(L2, L1, 3, 5)
C3

{('A', 'C', 'E'): {(35, 40)},
 ('A', 'E', 'C'): {(59, 62)},
 ('B', 'A', 'C'): {(46, 51), (57, 62)},
 ('B', 'A', 'E'): {(57, 61)},
 ('B', 'C', 'A'): set(),
 ('B', 'C', 'E'): {(37, 40)},
 ('B', 'E', 'A'): {(57, 60)},
 ('B', 'E', 'C'): {(57, 62)},
 ('C', 'A', 'E'): set(),
 ('C', 'B', 'A'): {(55, 60)},
 ('C', 'B', 'E'): {(55, 59)},
 ('C', 'E', 'A'): {(55, 60)},
 ('C', 'E', 'F'): {(38, 41), (50, 55)},
 ('C', 'F', 'A'): {(61, 66)},
 ('C', 'F', 'B'): set(),
 ('E', 'A', 'C'): {(58, 62)},
 ('E', 'C', 'A'): set(),
 ('E', 'C', 'F'): {(60, 63)},
 ('E', 'F', 'A'): {(31, 36)},
 ('E', 'F', 'C'): {(39, 43), (53, 56)},
 ('F', 'A', 'C'): set(),
 ('F', 'B', 'A'): set(),
 ('F', 'B', 'C'): set(),
 ('F', 'C', 'A'): set(),
 ('F', 'C', 'B'): {(54, 58)}}

In [20]:
L3 = createLargeEpisodeSet(C3, 2)
L3

{('B', 'A', 'C'): {(46, 51), (57, 62)},
 ('C', 'E', 'F'): {(38, 41), (50, 55)},
 ('E', 'F', 'C'): {(39, 43), (53, 56)}}

In [21]:
C4 = createCandidates(L3, L1, 4, 5)
C4

{}

In [22]:
def MinEpi(sequence, max_width, step, minFrequent):
    C1 = createC1(sequence, step)
    L1 = createLargeEpisodeSet(C1, minFrequent)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = createCandidates(L[k-2], L1, k, max_width)
        Lk = createLargeEpisodeSet(Ck, minFrequent)
        L.append(Lk)
        k += 1
    #remove empty last itemset from L
    if L[-1] == {}:
        L.pop()
    return L

In [23]:
L = MinEpi(sequence, max_width=5, step=1, minFrequent=2)
L

[{('E',): {(31, 32), (39, 40), (53, 54), (58, 59), (60, 61)},
  ('D',): {(32, 33), (44, 45), (48, 49), (67, 68)},
  ('F',): {(33, 34), (40, 41), (54, 55), (62, 63)},
  ('A',): {(35, 36), (47, 48), (59, 60), (65, 66)},
  ('B',): {(37, 38), (46, 47), (57, 58)},
  ('C',): {(38, 39), (42, 43), (50, 51), (55, 56), (61, 62)}},
 {('A', 'C'): {(35, 39), (47, 51), (59, 62)},
  ('A', 'D'): {(47, 49), (65, 68)},
  ('A', 'E'): {(35, 40), (59, 61)},
  ('B', 'A'): {(46, 48), (57, 60)},
  ('B', 'C'): {(37, 39), (46, 51), (57, 62)},
  ('B', 'E'): {(37, 40), (57, 59)},
  ('C', 'A'): {(55, 60), (61, 66)},
  ('C', 'B'): {(42, 47), (55, 58)},
  ('C', 'E'): {(38, 40), (50, 54), (55, 59)},
  ('C', 'F'): {(38, 41), (50, 55), (61, 63)},
  ('D', 'A'): {(32, 36), (44, 48)},
  ('E', 'A'): {(31, 36), (58, 60)},
  ('E', 'C'): {(39, 43), (53, 56), (60, 62)},
  ('E', 'F'): {(31, 34), (39, 41), (53, 55), (60, 63)},
  ('F', 'A'): {(33, 36), (62, 66)},
  ('F', 'B'): {(33, 38), (54, 58)},
  ('F', 'C'): {(40, 43), (54, 5

In [24]:
def isAlphaOccurInBeta(mob, mos_alpha): #mo_beta include (mo_beta_ts) and (mo_beta_ts + width2)
    for moa in mos_alpha:
        if mob[0] <= moa[0] and moa[1] <= mob[1]:
            return True
    return False

In [34]:
def calcConfidence(mos_beta, wid1, mos_alpha, wid2):
    denominator = 0.0
    numerator = 0
    for mob in mos_beta:
        #te - ts <= width1
        if mob[1] - mob[0] <= wid1:
            denominator += 1
            #number of mo_beta [mob.ts, mob.ts + wid2) contains mo_alpha is numerator
            if isAlphaOccurInBeta((mob[0], mob[0] + wid2), mos_alpha):
                numerator += 1
    if denominator == 0:
        return 0
    else:  
        return numerator/denominator

In [26]:
def calcSupport(mos_alpha, wid2):
    support = 0
    for moa in mos_alpha:
        #te - ts <= width2
        if moa[1] - moa[0] <= wid2:
            support += 1
    return support

In [41]:
def generateRules(L, max_width, step, minConf):
    #calculate time bound W
    time_bound = list(range(step, max_width, step))
    time_bound.append(max_width)
    #generate all rules
    for i in range(1, len(L)): #only get the sets with two or more items
        for episode in L[i]: #for each episode in a level
            for j in range(1, i+1): # i+1 equal to length of an episode, so j is the length of lhs
                lhsList = list(combinations(episode, j))
                for lhs in lhsList: #each left-hand-side
                    for rightWid in time_bound: #each time bound of rhs
                        for leftWid in time_bound: #each time bound of lhs
                            #index of L is the episode length-1
                            conf = calcConfidence(L[j-1][lhs], leftWid, L[i][episode], rightWid)
                            if conf >= minConf:
                                print(list(lhs), " [", leftWid, "] ==> ", list(episode), " [", rightWid, "] [", calcSupport(L[i][episode], rightWid), ", ", conf,"]", sep="")

In [42]:
generateRules(L, 5, 1, 0.9)

['B'] [1] ==> ['B', 'C'] [5] [3, 1.0]
['B'] [2] ==> ['B', 'C'] [5] [3, 1.0]
['B'] [3] ==> ['B', 'C'] [5] [3, 1.0]
['B'] [4] ==> ['B', 'C'] [5] [3, 1.0]
['B'] [5] ==> ['B', 'C'] [5] [3, 1.0]
['E'] [1] ==> ['E', 'F'] [5] [4, 1.0]
['E'] [2] ==> ['E', 'F'] [5] [4, 1.0]
['E'] [3] ==> ['E', 'F'] [5] [4, 1.0]
['E'] [4] ==> ['E', 'F'] [5] [4, 1.0]
['E'] [5] ==> ['E', 'F'] [5] [4, 1.0]
['B', 'A'] [2] ==> ['B', 'A', 'C'] [5] [2, 1.0]
['B', 'A'] [3] ==> ['B', 'A', 'C'] [5] [2, 1.0]
['B', 'A'] [4] ==> ['B', 'A', 'C'] [5] [2, 1.0]
['B', 'A'] [5] ==> ['B', 'A', 'C'] [5] [2, 1.0]
['C', 'E'] [2] ==> ['C', 'E', 'F'] [3] [1, 1.0]
['C', 'E'] [3] ==> ['C', 'E', 'F'] [3] [1, 1.0]
['C', 'E'] [2] ==> ['C', 'E', 'F'] [4] [1, 1.0]
['C', 'E'] [3] ==> ['C', 'E', 'F'] [4] [1, 1.0]
['C', 'E'] [2] ==> ['C', 'E', 'F'] [5] [2, 1.0]
['C', 'E'] [3] ==> ['C', 'E', 'F'] [5] [2, 1.0]
['E', 'F'] [2] ==> ['E', 'F', 'C'] [4] [2, 1.0]
['E', 'F'] [2] ==> ['E', 'F', 'C'] [5] [2, 1.0]
